# Computer Lab 2: Time Series Cross Validation

**In this computer lab practical you will learn**

* How to apply time series cross validation to select the best forecasting model.

> To keep things simple we will work with naive forecasting models.

---
* A video introducing the notebook and a walk through of exercise 1 is available https://bit.ly/ts_cross_val
* A youtube video to help with python's generators (optional) https://www.youtube.com/watch?v=2eiFCQ-YAf4

# Standard Imports

In [ ]:
import pandas as pd
import numpy as np
import sys

# Forecast-tools imports

In [ ]:
# if running in Google Colab install forecast-tools
if 'google.colab' in sys.modules:
    !pip install forecast-tools

In [ ]:
from forecast_tools.baseline import SNaive, Naive1
from forecast_tools.metrics import mean_absolute_error

# Exercise 1: Implementing rolling forecast origin time series cross validation.

# Exercise 1.1: Load the data

**Task**:

* Import monthly outpatient appointments time series. This can be found in **"out_appoints_mth.csv"**

**Hints:** 
* This is monthly data.  You can use the Monthly Start ('MS') frequency

In [ ]:
# your code here ...
url = 'https://raw.githubusercontent.com/health-data-science-OR/' \
        + 'hpdm097-datasets/master/out_appoints_mth.csv'

# Exercise 1.2. Create a rolling forecast origin in a loop.

In rolling forecast origin we iteratively move our model into the future.  Assume we have the following training data.

```python
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
``` 

If we initially train our model on the first 5 data points and make a 1 step forecast, then a rolling forecast origin look like this:

```python
[1] train => [1, 2, 3, 4, 5], test => [6]
[2] train => [1, 2, 3, 4, 5, 6], test => [7]
[3] train => [1, 2, 3, 4, 5, 6, 7], test => [8]
[4] train => [1, 2, 3, 4, 5, 6, 7, 8], test => [9]
[5] train => [1, 2, 3, 4, 5, 6, 7, 8, 9], test => [10]
``` 

If instead we had a forecast horizon of 2 then our splits would look like:
```python
[1] train => [1, 2, 3, 4, 5], test => [6, 7]
[2] train => [1, 2, 3, 4, 5, 6], test => [7, 8]
[3] train => [1, 2, 3, 4, 5, 6, 7], test => [8, 9]
[4] train => [1, 2, 3, 4, 5, 6, 7, 8], test => [9, 10]
``` 

There are different ways to implement this in Python.  The simplest is to use a for loop.

**Task:**
    
* Code a rolling forecast origin for-loop that returns a `train` and `test` in each iteration of a training data input.
* In each cv-fold the length of test should be equal to a parameter `h` that represents the forecast horizon
* In each iteration the length of the training set should increase by 1. 
* You will need to work out how many iterations you should make!  Be careful not to go over the end of the dataset!
* You will need to select a initial training set size.  This could be for example a third of the data.

**Hints:**
* Try working with the following data set initially.  Make sure it gives you the results you expect.

```python
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
``` 

* The data above is held in a python `list`, in practice you will be passing in a `pandas.Series` or a `np.ndarray` all of these are **array-like** and accept python slice notation for example:

```python
train_fold = train[:5]
```

* In the above you could vary `5` using variables in a for loop.

In [ ]:
# your code here ...

# Exercise 1.3: Calculate the average MAE of SNaive

Now that you have the basic structure of rolling forecast origin returning train-test splits you can use it to iteratively fit a model and calculate its forecast accuracy.  We will work with `SNaive` and the outpatients dataset.

**Task**:

* Modify your code from 1.2. to train and predict using a `Naive1` model 
* In each iteration calculate and store the mean absolute error (MAE) 
* At the end of the loop calculate the mean cross validation score.

**Hints**:
* Declare a python list called `scores` and append the MAE of each split during the loop.
* This is monthly data so Seasonal Naive requires a parameter of 12.
* Don't forget to set the size of the initial training data.  E.g. half or a third of the dataset.

In [ ]:
# your code goes here ...

# Exercise 1.4: Refactoring your code as a generator

No matter which forecasting approach you use, you will do a lot of cross validation in practice.  To help, it is good to aim to get all of your code into a reusable format and start builing your own utility functions. For cross-validation there are a number of ways to do it.  One option is to convert your for-loop into a **generator**. 

**Task:**
* Refactor your code from 1.2 into a generator.  

**Hints:**
* Generators make use of the **yield** statement. At the end of each iteration you need to yield a tuple that contains (train, test).  
* Assuming you call your generator `rolling_forecast_origin`, The code that would use your generator could look like:

```python
train = np.arange(24)
model = SNaive(12)
cv_rolling = rolling_forecast_origin(train, min_train_size=5, horizon=2)
scores = []

for train_fold, test_fold in cv_rolling:
    model.fit(train)
    preds = model.predict(horizon)
    score = mean_absolute_error(test_fold, preds)
    scores.append(score)
     
print(np.array(scores).mean())
```

* The advantage of splittng the implementation of logic that does the splitting and the forecasting is that you can reuse `rolling_forecast_origin` with any forecasting method regardless of its interface.

In [ ]:
# your code here

# Exercise 2: Choosing the best naive model

We will work with ED reattendance dataset from the previous computer lab.  Your task is to select the best naive model.  Last time we used a simple holdout sample, this time we will use cross validation.

**Task:**

* Load the ED reattendance dataset 'ed_reattends.csv' (url provide below)
* Use a forecast horizon of 6 months with an initial training size of 1/3 of the dataset.
* Test all Naive models: `Naive1`, `SNaive`, `Average`, `Drift`, `EnsembleNaive`
* Select the forecast with the lowest mean absolute error.

**Hints:**
* This is monthly data - remember to seasonaly adjust 
* forecast-tools provides a convenience function to load all naive models

```python
>>> from forecast_tools.baseline import baseline_estimators
>>> estimators = baseline_estimators(seasonal_period=12)
>>> estimators

{'NF1': Naive1(),
 'SNaive': SNaive1(period=12),
 'Average': Average(),
 'Drift': Drift(),
 'Ensemble': EnsembleNaive(seasonal_period=12)}
```


In [ ]:
# your code here ...
url = 'https://raw.githubusercontent.com/health-data-science-OR/' \
        + 'hpdm097-datasets/master/ed_reattend.csv'

# Exercise 3: using forecast-tools built in cross-validation options

`forecast-tools` provides some optimised tools to help you with cross validation in the `forecast_tools.model_selection` module.

```python
>>> from forecast_tools.model_selection import (rolling_forecast_origin, 
                                                sliding_window)
```

These functions are implementations of Rolling Forecast Origin and Sliding Window splitting of the time series.  For example, for a rolling forecast origin split

```python
>>> min_train_size = len(reattend_rate) // 3
>>> cv = rolling_forecast_origin(train=reattend_rate, 
                                 min_train_size=min_train_size, 
                                 horizon=6, 
                                 step=1)
```

Notice that `rolling_forecast_origin` has an additional parameter `step`. If this is set to greater than 1 then this introduces a gap between folds.  This might used for instance if the time series is very large or if there is a desire to seperate the folds to reduce their similarity.

`forecast-tools` also contains utility functions to either return a cv scores or cv predictions.

```python

>>> from forecast_tools.model_selection import (cross_validation_score, 
                                                cross_validation_folds)
```

The first of these functions returns a numpy array of the forecast errors for each fold and the latter returns a numpy array the train, test and y_preds in each fold.

```python
>>> from forecast_tools.metrics import root_mean_squared_error
>>> nf1 = Naive1()
>>> scores = cross_validation_score(model=nf1, cv=cv, 
                                    metric=root_mean_squared_error,
                                    n_jobs=-1)
>>> print(scores.mean())

4.074747242185809
```

`cross_validation_score` accepts any metric with a `metric(y_true, y_pred)` pattern. So you could use `mean_absolute_error` or another of the metrics in place of `root_mean_squared_error`.  The parameters `n_jobs=-1` means that the cross validation is conducted in parallel.  You can also assess multiple forecast horizons in one go by using the horizons parameter.  For example if you wanted to report the cv scores at 1-6 month intervals set `horizons=[1, 2, 3, 4, 5, 6]`

```python
>>> nf1 = Naive1()
>>> horizons = [1, 2, 3, 4, 5, 6]
>>> scores = cross_validation_score(model=nf1, cv=cv, 
                                    metric=root_mean_squared_error, 
                                    horizons=horizons,
                                    n_jobs=-1)

>>> print(pd.DataFrame(scores, columns=horizons).mean())

1    3.052419
2    3.510499
3    3.605947
4    3.759665
5    3.944258
6    4.074747
dtype: float64
```

# Exercise 3.1 Repeat exercise 2 using forecast-tools.

**Task**:
* Load the ED reattendance dataset 'ed_reattends.csv'
* Use a forecast horizon of 6 months with an initial training size of 1/3 of the dataset.
* Test all Naive models: `Naive1`, `SNaive`, `Average`, `Drift`, `EnsembleNaive`
* Select the forecast with the lowest mean absolute error.
* Use forecast-tools to do the cross validation.

In [ ]:
# your code here ...

## Exercise 3.2: cross validation of multiple forecast horizons

**Task:** 
    
* Using forecast-tools and the ed_reattend.csv data how do Naive1 and EnsembleNaive compare over forecast horizons of 1 to 6 months?

In [ ]:
# your code here ...

# Exercise 3.3: automatically selecting the best naive method

`forecast-tools` provides an `auto_naive()` function to automatically select the 'best' naive method to serve as a benchmark.  This works via cross-validation.  The tool has a number of settings, but for practical purposes most of these can be left to the defaults.  

```python
>>> from forecast_tools.model_selection import auto_naive
>>> benchmark = auto_naive(y_train=reattend_rate, horizon=6, seasonal_period=12)
>>> print(benchmark)

{'model': EnsembleNaive(seasonal_period=12), 'mae': 3.091713319664855}
```

The function returns a dict that contains both the best model and the metric of interest (by default the MAE).

**Task:**
* Use `auto_naive` to select a benchmark model for the outpatient appointments dataset 'out_appoints_mth.csv'
* Try running the function using a different metric such as MAPE.  Use `help(auto_naive)` to see how parameters work.

In [ ]:
# your code here